In [4]:
import argparse
import mysql.connector
from astropy.table import Table

In [51]:
'''database setup commands'''

def get_db_connection():
    return mysql.connector.connect(
        host='localhost',
        user='nana',
        database='stars_db')

def execute_query_and_close(query):
    conn = get_db_connection()
    conn.cursor().execute(query)
    conn.commit()
    conn.cursor().close()
    conn.close()
    
def load_star_table():
    filename = "KICids.csv"
    foo = Table.read(filename, format = "ascii.csv")
    kic_ids = [f"KIC{f:09d}" for f in foo['ID']]
    query = "DELETE FROM star; INSERT INTO star (star_id) VALUES ('" + "'),('".join(kic_ids) + "');"
    execute_query_and_close(query)

def load_dataset_table():
    query = """DELETE FROM dataset; 
    INSERT INTO dataset (dataset_id, description) 
    VALUES ('Kepler_long', 'Long-cadence data from the NASA Kepler Mission');"""
    execute_query_and_close(query)
    

In [52]:
load_dataset_table()

In [47]:
load_star_table()

In [ ]:
'''database update commands'''

def track_status(star_id, dataset_id, status, process_id=None):
    conn = mysql.connector.connect(
        host='localhost',
        user='nana',
        database='stars_db'
    )
    cursor = conn.cursor()
    if status == 'started':
        cursor.execute(
            """
            INSERT INTO task (star_id, dataset_id, started)
            VALUES (%s, %s, NOW())
            ON DUPLICATE KEY UPDATE started=NOW()
            """,
            (star_id, dataset_id)
        )
        print(f"Started task for star_id={star_id}, dataset_id={dataset_id}")

    elif status == 'finished':
        cursor.execute(
            "UPDATE task SET finished=NOW() WHERE star_id=%s AND dataset_id=%s",
            (star_id, dataset_id)
        )
        print(f"Finished task for star_id={star_id}, dataset_id={dataset_id}")
    conn.commit()
    cursor.close()
    conn.close()

def record_star(star_id, dataset_id):
    conn = mysql.connector.connect(
        host='localhost',
        user='nana',
        database='stars_db'
    )
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO star (star_id, dataset_id) VALUES (%s, %s) "
        "ON DUPLICATE KEY UPDATE dataset_id=%s",
        (star_id, dataset_id, dataset_id)
    )
    conn.commit()
    cursor.close()
    conn.close()

 
# def record_mode(star_id, dataset_id, frequency):

def main():
    parser = argparse.ArgumentParser(description="Process a star.")
    parser.add_argument("star_id", type=str)
    parser.add_argument("--dataset_id", type=str, default="Kepler_LC")
    args = parser.parse_args()

    star_id = args.star_id
    dataset_id = args.dataset_id

    track_status(star_id, dataset_id, 'started')

    success = find_modes_in_star(star_id, plots=False, save=False, max_peaks=100)

    if success:  
        record_star(star_id, dataset_id)
        #record_mode(star_id, dataset_id)

    track_status(star_id, dataset_id, 'finished')


if __name__ == "__main__":
    main()
